In [1]:
import numpy as np
import gym
from collections import deque
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


#### 经验回放池：

In [4]:
class ReplayBuffer:
    ''' 经验回放池 '''
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)        # 先进先出队列

    def add(self, state, action, reward, next_state, done):  
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):  
        transitions = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = zip(*transitions)
        return np.array(state), np.array(action), reward, np.array(next_state), done

    def size(self): 
        return len(self.buffer)

#### DQN:
1. DQN 算法概述
DQN 是一种深度强化学习算法，结合 Q-Learning 和 深度神经网络，用于处理高维状态空间（如图像或连续状态）。由 DeepMind 在 2013 和 2015 年提出，广泛应用于 Atari 游戏等任务。DQN 使用神经网络近似 Q 值函数 $Q(s, a; \theta)$，通过优化时序差分（TD）误差学习最优策略。

2. 数学原理
2.1 强化学习与马尔可夫决策过程 (MDP)
- **强化学习建模为 MDP $(S, A, P, R, \gamma)$**：
状态 $S$：RollingBall 的 $[x, y, v_x, v_y]$。
动作 $A$：25 个离散动作（Hashposotion.py）。
转移概率 $P(s' | s, a)$：由 GridWorld.py 物理规则隐式定义。
奖励 $R(s, a, s')$：-2.0（每步）、-10.0（撞墙）、+300.0（目标）。
折扣因子 $\gamma$：通常 0.99。

- **目标：最大化累积折扣奖励**：
$$
J(\pi) = \mathbb{E}{\pi} \left[ \sum{t=0}^\infty \gamma^t R(s_t, a_t, s_{t+1}) \right]
$$
平稳分布 $d_\pi(s)$，满足 $d_\pi^T P_\pi = d_\pi^T$，描述策略 $\pi$ 下的长期状态访问概率。

2.2 Q-Learning 与 Bellman 方程
Q 值函数 $Q^\pi(s, a)$ 表示状态 $s$ 下执行动作 $a$，然后按策略 $\pi$ 行动的预期累积奖励：
$$Q^\pi(s, a) = \mathbb{E}{\pi} \left[ R(s, a, s') + \gamma \sum{a'} \pi(a'|s') Q^\pi(s', a') \right]$$
最优 Q 值函数 $Q^*(s, a)$ 满足 Bellman 最优方程（《Chapter 8》, Box 7.5）：
$$Q^(s, a) = \mathbb{E} \left[ R(s, a, s') + \gamma \max_{a'} Q^(s', a') \right]$$
Q-Learning 更新：
$$Q(s, a) \leftarrow Q(s, a) + \alpha \left[ R(s, a, s') + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]$$
RollingBall 的连续状态使 Q 表格不可行，需函数逼近。
2.3 DQN 的函数逼近
DQN 使用神经网络 $Q(s, a; \theta)$ 近似 $Q^*(s, a)$。

表格法：存储所有 $Q(s, a)$，对连续状态不可行。
函数逼近：用参数 $\theta$ 表示 $Q(s, a; \theta)$，存储效率高，泛化能力强。

RollingBall 配置：

输入：4 维状态 $[x, y, v_x, v_y]$。
输出：25 维 Q 值向量。
网络：全连接网络（2-3 层，128 神经元，ReLU 激活）。

DQN 优化 Bellman 误差：
$$J(\theta) = \mathbb{E} \left[ \left( R + \gamma \max_{a'} Q(S', a'; \theta) - Q(S, A; \theta) \right)^2 \right]$$
直接优化不稳定，需以下技术。
3. DQN 的关键技术及数学推导
3.1 经验回放（Experience Replay）
问题：经验 $(s, a, r, s', \text{done})$ 具有时间相关性，违背神经网络训练的 i.i.d. 假设。
解决方案：回放池存储经验，随机采样批量数据。强调均匀采样确保状态-动作对 $(S, A)$ 近似均匀分布。
数学原理：

回放池 $\mathcal{D}$ 存储 $(s, a, r, s', \text{done})$。
采样 $N$ 个经验，损失为：

$$L(\theta) = \frac{1}{N} \sum_{i=1}^N \left[ r_i + \gamma (1 - \text{done}i) \max{a'} Q(s'_i, a'; \theta) - Q(s_i, a_i; \theta) \right]^2$$

3.2 目标网络（Target Network）
问题：目标值 $r + \gamma \max_{a'} Q(s', a'; \theta)$ 随 $\theta$ 变化，训练不稳定。
解决方案：目标网络 $Q(s, a; \theta^-)$，参数 $\theta^-$ 定期复制。《Chapter 8》（Section 8.4.1）描述固定 $\theta^-$ 简化梯度计算：
$$L(\theta) = \mathbb{E} \left[ \left( r + \gamma \max_{a'} Q(s', a'; \theta^-) - Q(s, a; \theta) \right)^2 \right]$$
数学推导：

目标值：

$$y_i = r_i + \gamma (1 - \text{done}i) \max{a'} Q(s'_i, a'; \theta^-)$$

梯度：

$$\nabla_\theta L(\theta) = -\frac{2}{N} \sum_{i=1}^N \left[ y_i - Q(s_i, a_i; \theta) \right] \nabla_\theta Q(s_i, a_i; \theta)$$

每 $k$ 步（如 100），$\theta^- \leftarrow \theta$。


3.3 ε-贪婪策略
问题：需平衡探索与利用。
解决方案：以概率 $\epsilon$ 随机选择动作，否则选择 $\arg\max_a Q(s, a; \theta)$。《Chapter 8》（Box 8.1）提到探索策略（如 ε-贪婪）确保平稳分布唯一。
数学原理：

策略：

$$\pi(a|s) = \begin{cases}\frac{\epsilon}{|A|} + (1 - \epsilon) \cdot \mathbb{I}[a = \arg\max_{a'} Q(s, a'; \theta)] & \text{if } a = \arg\max_{a'} Q(s, a'; \theta) \\frac{\epsilon}{|A|} & \text{otherwise}\end{cases}$$

衰减：

$$\epsilon_t = \epsilon_{\text{end}} + (\epsilon_{\text{start}} - \epsilon_{\text{end}}) \exp(-t / \tau)$$
与 RollingBall 的结合：

env.action_space.sample() 实现随机动作。
衰减 $\epsilon$（如 1.0 到 0.1）鼓励目标探索。

4. DQN 算法流程
综合《Chapter 8》（Algorithm 8.3）和 hrl.boyuai.com：

初始化：

在线网络 $Q(s, a; \theta)$，目标网络 $Q(s, a; \theta^-)$，$\theta^- \leftarrow \theta$。
回放池 $\mathcal{D}$，容量 capacity。
超参数：学习率 $\alpha$，折扣因子 $\gamma$，$\epsilon$ 参数。

交互：

按 ε-贪婪选择 $a$。
执行 $a$，获取 $r, s', \text{done}$。
存储 $(s, a, r, s', \text{done})$。

优化：

采样批量经验。

目标值：
$$y_i = r_i + \gamma (1 - \text{done}i) \max{a'} Q(s'_i, a'; \theta^-)$$

损失：
$$L(\theta) = \frac{1}{N} \sum_{i=1}^N \left[ y_i - Q(s_i, a_i; \theta) \right]^2$$

更新：

每 $k$ 步，$\theta^- \leftarrow \theta$。
衰减 $\epsilon_t$。

5. 局限与改进

Q 值过估计：

$\max_{a'} Q(s', a'; \theta^-)$ 可能高估。《Chapter 10》（Section 10.2）提到基线减少方差，DQN 用 Double DQN：
$$y_i = r_i + \gamma (1 - \text{done}_i) Q(s'i, \arg\max{a'} Q(s'_i, a'; \theta); \theta^-)$$

稀疏奖励：RollingBall 的 +300.0 奖励稀疏，需奖励整形。

离散动作：DQN 限于离散动作

In [5]:
class Q_Net(torch.nn.Module):
    ''' Q 网络是一个两层 MLP, 用于 DQN 和 Double DQN '''
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = torch.nn.Linear(hidden_dim, output_dim)
        # 初始化权重
        self._init_weights()
    
    def _init_weights(self):
        """使用Kaiming初始化权重，适合激活函数为Relu"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                # 使用 Kaiming 均匀初始化，指定 nonlinearity='relu'
                nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')
                # 偏置置零
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
                    
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        return self.fc3(x)

1. self.bn1 = nn.BatchNorm1d(hidden_dim) 是定义一个一维批归一化（Batch Normalization）层的操作。
- **核心功能**针对全连接层（Linear）或一维卷积层（Conv1D）的输出.对每个隐藏层神经元的输出进行标准化处理：
    $$
    \hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} \quad \text{(标准化)}
    $$
    $$
    y_i = \gamma \hat{x}_i + \beta \quad \text{(可学习缩放和平移)}
    $$
  其中：
$μ_B$是该神经元在当前batch上的均值,$σ_B^2$是方差,γ(weight)和β(bias)是可学习的参数。

- **在DQN中的具体作用:**

    正向传播时：标准化过程计算当前batch中每个神经元的均值和方差，对128个神经元的输出独立标准化，通过γ和β保留网络的表达能力

    反向传播时：自动更新当前batch的$μ_B$和$σ_B^2$、可学习参数γ和β、全局统计量（用于推理的running_mean和running_var）

- **数据流维度变化：**

    输入: [batch_size, 4] 
    
    fc1: [batch_size, 128] 

    bn1: [batch_size, 128] (标准化后)

    fc2: [batch_size, 128]

    输出: [batch_size, 25]
    通过这种设计，BatchNorm1d能有效提升DQN在RollingBall环境中的训练稳定性和收敛速度。

---

In [6]:
class DQN(torch.nn.Module):
    ''' DQN算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, action_range, lr, gamma, epsilon, device, tau=0.001, seed=None):
        super().__init__()
        self.action_dim = action_dim
        self.state_dim = state_dim
        self.hidden_dim = hidden_dim
        self.action_range = action_range        # action 取值范围
        self.gamma = gamma                      # 折扣因子
        self.epsilon = epsilon                  # epsilon-greedy
        self.tau = tau      # 目标网络更新频率
        self.count = 0                          # Q_Net 更新计数
        self.rng = np.random.RandomState(seed)  # agent 使用的随机数生成器
        self.device = device                
        
        # Q 网络
        self.q_net = Q_Net(state_dim, hidden_dim, action_range).to(device)  
        # 目标网络
        self.target_q_net = Q_Net(state_dim, hidden_dim, action_range).to(device)
        # 使用Adam优化器
        self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=lr)
        
    def max_q_value_of_given_state(self, state):
        state = torch.tensor(state, dtype=torch.float).to(self.device)
        return self.q_net(state).max().item()
        
    def take_action(self, state):  
        ''' 按照 epsilon-greedy 策略采样动作 '''
        if self.rng.random() < self.epsilon:
            action = self.rng.randint(self.action_range)
        else:
            state = torch.tensor(state, dtype=torch.float).to(self.device)
            action = self.q_net(state).argmax().item()
        return action

    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'], dtype=torch.float).to(self.device)                             # (bsz, state_dim)
        next_states = torch.tensor(transition_dict['next_states'], dtype=torch.float).to(self.device)                   # (bsz, state_dim)
        actions = torch.tensor(transition_dict['actions'], dtype=torch.int64).view(-1, 1).to(self.device)               # (bsz, act_dim)
        rewards = torch.tensor(transition_dict['rewards'], dtype=torch.float).view(-1, 1).to(self.device).squeeze()     # (bsz, )
        dones = torch.tensor(transition_dict['dones'], dtype=torch.float).view(-1, 1).to(self.device).squeeze()         # (bsz, )

        q_values = self.q_net(states).gather(dim=1, index=actions).squeeze()                # (bsz, )
        max_next_q_values = self.target_q_net(next_states).max(axis=1)[0]                   # (bsz, )
        q_targets = rewards + self.gamma * max_next_q_values * (1 - dones)                  # (bsz, )

        dqn_loss = torch.mean(F.mse_loss(q_values, q_targets))  
        self.optimizer.zero_grad()                                                         
        dqn_loss.backward() 
        self.optimizer.step()
        
        # 软更新目标网络参数
        for target_param, q_param in zip(self.target_q_net.parameters(), self.q_net.parameters()):
            target_param.data.copy_(self.tau * q_param.data + (1.0 - self.tau) * target_param.data)


#### DoubleDQN:

In [7]:
class DoubleDQN(DQN):
    ''' Double DQN算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, action_range, lr, gamma, epsilon, device, tau=0.001, seed=None):
        super().__init__(state_dim, hidden_dim, action_dim, action_range, lr, gamma, epsilon, device, tau, seed)
    
    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'], dtype=torch.float).to(self.device)                             # (bsz, state_dim)
        next_states = torch.tensor(transition_dict['next_states'], dtype=torch.float).to(self.device)                   # (bsz, state_dim)
        actions = torch.tensor(transition_dict['actions'], dtype=torch.int64).view(-1, 1).to(self.device)               # (bsz, act_dim)
        rewards = torch.tensor(transition_dict['rewards'], dtype=torch.float).view(-1, 1).to(self.device).squeeze()     # (bsz, )
        dones = torch.tensor(transition_dict['dones'], dtype=torch.float).view(-1, 1).to(self.device).squeeze()         # (bsz, )
        # Double DQN：主网络选择动作，目标网络估计Q值
        q_values = self.q_net(states).gather(dim=1, index=actions).squeeze()                # (bsz, )
        # 使用Q网络估计最优动作（[0]取最优值，[1]取最优值的索引）
        max_actions_index = self.q_net(next_states).max(axis=1)[1]
        # 由目标网络计算Q值
        max_next_q_values = self.target_q_net(next_states).gather(dim=1, index = max_actions_index.unsqueeze(1)).squeeze()                   # (bsz, )
        q_targets = rewards + self.gamma * max_next_q_values * (1 - dones)                  # (bsz, )

        dqn_loss = torch.mean(F.mse_loss(q_values, q_targets))  
        self.optimizer.zero_grad()                                                         
        dqn_loss.backward() 
        self.optimizer.step()
        
        # 软更新目标网络参数
        for target_param, q_param in zip(self.target_q_net.parameters(), self.q_net.parameters()):
            target_param.data.copy_(self.tau * q_param.data + (1.0 - self.tau) * target_param.data)

#### **states 和 next_states**

- **初始形状**：`transition_dict['states']` 是一个长度为 $ T $ 的列表，其中 $ T $ 是轨迹的长度（即一个回合的步数）。列表中的每个元素是一个状态，形状为 `(state_dim,)`。
  - **解释**：在强化学习中，`state` 表示环境在某个时间步的状态。例如，在 `RollingBall` 环境中，状态可能是 `[x, y, vx, vy]`，因此 `state_dim = 4`。如果轨迹长度 $ T = 100 $，则 `transition_dict['states']` 是一个包含 100 个状态的列表，每个状态的形状为 `(4,)`。
  - **示例**：`transition_dict['states'] = [[0.5, 0.3, 1.0, -0.2], [0.6, 0.4, 0.9, -0.3], ..., [1.2, 0.8, 0.1, 0.0]]`。

- **转换后形状**：通过 `torch.tensor(transition_dict['states'], dtype=torch.float)` 转换为 PyTorch 张量，形状变为 `(T, state_dim)`，即 `(bsz, state_dim)`，其中 $ bsz = T $。
  - **解释**：
    - `torch.tensor` 将列表转换为张量，列表的每个元素（状态）成为张量的一行。
    - 转换后，张量的形状是 `(T, state_dim)`，其中：
      - 第一维 $ T $（即 $ bsz $）表示批量大小（batch size），这里等于轨迹长度。
      - 第二维 `state_dim` 是每个状态的维度。
    - **示例**：若 $ T = 100 $，`state_dim = 4`，则 `states` 的形状为 `(100, 4)`，表示 100 个状态，每个状态有 4 个特征。
    - **张量内容**：`tensor([[0.5, 0.3, 1.0, -0.2], [0.6, 0.4, 0.9, -0.3], ..., [1.2, 0.8, 0.1, 0.0]])`。

- **对于 `next_states` 的分析**：
  - `transition_dict['next_states']` 也是一个长度为 $ T $ 的列表，每个元素是一个下一状态，形状为 `(state_dim,)`。
  - 转换过程与 `states` 相同，形状从列表转换为 `(T, state_dim)`。
  - **示例**：`next_states` 的形状同样为 `(100, 4)`，表示 100 个下一状态。

- **总结**：
  - `states` 和 `next_states` 的形状变化：
    - 初始：长度为 $ T $ 的列表，每个元素形状为 `(state_dim,)`。
    - 转换后：`(T, state_dim)`，即 `(bsz, state_dim)`，其中 $ bsz = T $。
  - **作用**：这种形状适配批量计算，适合输入到策略网络（Actor）或价值网络（Critic）中，例如计算 $\pi_\theta(a|s)$ 或 $V(s)$。

---

#### **actions**

- **初始形状**：`transition_dict['actions']` 是一个长度为 $ T $ 的列表，每个元素是一个整数，表示离散动作的索引（例如 0 到 `action_range-1`）。
  - **解释**：在强化学习中，`actions` 是代理在每个时间步选择的动作。在离散动作空间中，每个动作是一个整数。例如，在 `RollingBall` 环境中，动作空间被离散化为 `action_range = 100`（通过 `DiscreteActionWrapper` 和 `FlattenActionSpaceWrapper`），因此每个动作是 0 到 99 之间的整数。
  - **示例**：若 $ T = 100 $，`transition_dict['actions'] = [0, 1, 2, ..., 99]`，是一个长度为 100 的列表。

- **转换后形状**：
  - **第一步**：`torch.tensor(transition_dict['actions'], dtype=torch.int64)` 转换为张量，形状为 `(T,)`。
    - **解释**：`torch.tensor` 将列表转换为一维张量，`dtype=torch.int64` 指定数据类型为 64 位整数，适合离散动作。
    - **示例**：`tensor([0, 1, 2, ..., 99])`，形状为 `(100,)`。
  - **第二步**：`.view(-1, 1)` 重塑张量形状，`-1` 表示自动计算维度（这里是 $ T $），`1` 表示新增一个维度，形状变为 `(T, 1)`，即 `(bsz, act_dim)`，其中 `act_dim = 1`。
    - **解释**：`view(-1, 1)` 将一维张量重塑为二维张量，新增的维度表示动作的维度（`act_dim = 1`），因为每个动作是一个标量。
    - **示例**：形状从 `(100,)` 变为 `(100, 1)`，如 `tensor([[0], [1], [2], ..., [99]])`。
  - **最终形状**：`(bsz, act_dim)`，其中 $ bsz = T $，`act_dim = 1`。

- **总结**：
  - `actions` 的形状变化：
    - 初始：长度为 $ T $ 的列表，每个元素是一个整数。
    - 转换后：`(T,)` -> `(T, 1)`（通过 `view`），即 `(bsz, act_dim)`，`act_dim = 1`。
  - **作用**：`(T, 1)` 的形状适配批量操作，例如用于从策略网络的输出中选择动作概率（如 `probs[range(bsz), actions]`）。

---

#### **rewards 和 dones**

- **初始形状**：`transition_dict['rewards']` 和 `transition_dict['dones']` 都是长度为 $ T $ 的列表。
  - **对于 `rewards`**：每个元素是一个浮点数，表示时间步的奖励。
    - **示例**：`transition_dict['rewards'] = [1.0, 0.5, -0.2, ..., 3.0]`，长度为 100。
  - **对于 `dones`**：每个元素是一个布尔值（`True` 或 `False`），表示时间步是否终止（例如达到目标或超时）。
    - **示例**：`transition_dict['dones'] = [False, False, ..., True]`，长度为 100。

- **转换后形状**：
  - **第一步**：`torch.tensor(transition_dict['rewards'], dtype=torch.float)` 转换为张量，形状为 `(T,)`。
    - **解释**：`torch.tensor` 将列表转换为一维张量，`dtype=torch.float` 指定数据类型为浮点数。
    - **示例**：`tensor([1.0, 0.5, -0.2, ..., 3.0])`，形状为 `(100,)`。
  - **第二步**：`.view(-1, 1)` 重塑张量形状，`-1` 表示自动计算维度（这里是 $ T $），`1` 表示新增一个维度，形状变为 `(T, 1)`。
    - **解释**：`view(-1, 1)` 将一维张量重塑为二维张量，新增的维度是为了适配可能的矩阵运算。
    - **示例**：形状从 `(100,)` 变为 `(100, 1)`，如 `tensor([[1.0], [0.5], [-0.2], ..., [3.0]])`。
  - **第三步**：`.squeeze()` 移除所有维度为 1 的轴，形状从 `(T, 1)` 回到 `(T,)`。
    - **解释**：`squeeze()` 移除单维度，恢复为一维张量，简化后续计算。
    - **示例**：形状从 `(100, 1)` 变为 `(100,)`，如 `tensor([1.0, 0.5, -0.2, ..., 3.0])`。
  - **最终形状**：`(bsz,)`，其中 $ bsz = T $。
  - **对于 `dones` 的转换**：
    - 转换过程与 `rewards` 相同。
    - `torch.tensor` 将布尔值转换为浮点数（`True` -> 1.0，`False` -> 0.0）。
    - **示例**：`tensor([0.0, 0.0, ..., 1.0])`，形状为 `(100,)`。

- **总结**：
  - `rewards` 和 `dones` 的形状变化：
    - 初始：长度为 $ T $的列表。
    - 转换后：`(T,)` -> `(T, 1)`（通过 `view`） -> `(T,)`（通过 `squeeze`），即 `(bsz,)`。
  - **作用**：`(bsz,)` 的形状适配批量计算，例如计算折扣回报 $G_t$ 或 TD 误差。

---

### 综合解释

- **批量计算的意义**：
  - 上述形状变化的目的是将轨迹数据转换为张量，支持批量计算。
  - 在 Actor-Critic 或 REINFORCE 算法中，批量计算可以一次性处理整个轨迹（长度为 $ T $），提高效率。例如：
    - `states` 和 `next_states` 用于批量输入策略网络或价值网络。
    - `actions` 用于从策略网络输出中选择动作概率。
    - `rewards` 和 `dones` 用于计算折扣回报或 TD 目标。

- **为何使用 `view` 和 `squeeze`**：
  - `view(-1, 1)`：将一维张量重塑为二维张量，新增的维度适配矩阵运算。例如，在计算损失时，可能需要张量形状一致。
  - `squeeze()`：在不需要额外维度时，移除单维度，简化张量形状，方便后续操作（如标量运算）。

- **张量设备**：
  - `.to(self.device)` 确保所有张量在同一设备（CPU 或 GPU）上，与策略网络和价值网络的设备一致，避免设备不匹配的错误。

---

### 示例

假设轨迹长度 $ T = 100 $，`state_dim = 4`（`RollingBall` 环境），`action_range = 100`：

- **`states` 和 `next_states`**：
  - 初始：`transition_dict['states'] = [[0.5, 0.3, 1.0, -0.2], ..., [1.2, 0.8, 0.1, 0.0]]`，长度为 100。
  - 转换后：`(100, 4)`。
- **`actions`**：
  - 初始：`transition_dict['actions'] = [0, 1, 2, ..., 99]`，长度为 100。
  - 转换后：`(100,)` -> `(100, 1)`。
- **`rewards` 和 `dones`**：
  - 初始：`transition_dict['rewards'] = [1.0, 0.5, -0.2, ..., 3.0]`，`transition_dict['dones'] = [False, False, ..., True]`，长度为 100。
  - 转换后：`(100,)` -> `(100, 1)` -> `(100,)`。

这些张量将用于后续的梯度计算和参数更新，确保高效的批量操作。

#### Dueling DQN:

Dueling DQN（Dueling Deep Q-Network）是一种改进的深度强化学习算法，通过优化神经网络结构来提升性能，尤其在动作空间较大或状态复杂的情况下。研究表明，Dueling DQN 通过将 Q 值函数分解为状态价值和动作优势两个部分，能够更高效地学习策略，减少 Q 值过估计，并提高训练稳定性。以下是其数学原理和各个方面的简要概述，适合初学者理解。

- **关键点**：
  - Dueling DQN 将 Q 值分解为状态价值 $ V(s) $ 和动作优势 $ A(s, a) $，提高学习效率。
  - 它使用两个神经网络分支：一个估计状态价值，一个估计动作优势。
  - 损失函数与标准 DQN 相同，但作用于分解后的 Q 值。
  - 相比标准 DQN，Dueling DQN 在动作相似或动作空间大的环境中表现更优。
  - 没有显著争议，但其效果依赖于环境特性，可能不总是优于标准 DQN。

###### 什么是 Dueling DQN？
Dueling DQN 是标准 DQN 的改进版本。标准 DQN 使用神经网络直接预测每个动作的 Q 值（即在状态 $ s $ 下选择动作 $ a $ 的预期回报）。Dueling DQN 则将 Q 值分解为两部分：状态价值（表示状态的好坏）和动作优势（表示动作的相对优劣）。这种分解让模型能更高效地学习状态的重要性，尤其在动作选择影响较小的环境中。

###### 为什么更有效？
在某些环境中（如 Atari 游戏），许多动作的回报差异不大。Dueling DQN 通过单独学习状态价值，减少对每个动作的逐一评估，从而更快地学习到好的策略。它还通过减去平均优势值来稳定训练，减少 Q 值过估计的问题。

###### 如何实现？
Dueling DQN 使用一个共享的特征提取层（例如全连接层或卷积层），然后分为两个分支：一个输出状态价值 $ V(s) $，另一个输出动作优势 $ A(s, a) $。最终 Q 值通过公式组合：
$$
Q(s, a) = V(s) + \left( A(s, a) - \frac{1}{|\mathcal{A}|} \sum_{a'} A(s, a') \right)
$$
训练过程与标准 DQN 类似，使用经验回放和目标网络来优化损失函数。

###### 适用场景
Dueling DQN 特别适合动作空间较大或状态复杂的任务，例如你的 `RollingBall` 环境（100 个离散动作）。它能更高效地处理动作选择不敏感的状态，提高学习速度。

---

##### Dueling DQN 的数学原理

1. 背景与动机

**标准 DQN 的局限性**：
- **直接估计 Q 值**：标准 DQN 使用神经网络直接估计 $ Q(s, a) $，输出每个动作的 Q 值。这在动作空间较大时计算量大。
- **动作无关性**：在某些环境中（如 Atari 游戏），许多动作对长期回报的影响相似，DQN 无法有效利用这一特性，导致学习效率低下。
- **Q 值过估计**：由于最大化操作（$\max_a Q(s', a')$），DQN 可能高估 Q 值，影响策略稳定性。

**Dueling DQN 的创新**：
- Dueling DQN 将 Q 值函数分解为状态价值 $ V(s) $ 和动作优势 $ A(s, a) $，通过两个神经网络分支分别学习：
  - $ V(s) $：表示状态 $ s $ 的整体价值，与动作无关。
  - $ A(s, a) $：表示在状态 $ s $ 下选择动作 $ a $ 相对于平均动作价值的优势。
- Q 值公式：
  $$
  Q(s, a) = V(s) + A(s, a)
  $$
- 为确保可识别性（避免 $ V(s) $ 和 $ A(s, a) $ 的分解不唯一），通常减去平均优势值：
  $$
  Q(s, a; \theta, \alpha, \beta) = V(s; \theta, \beta) + \left( A(s, a; \theta, \alpha) - \frac{1}{|\mathcal{A}|} \sum_{a'} A(s, a'; \theta, \alpha) \right)
  $$
  - $ \theta $：共享特征提取层的参数。
  - $ \beta $：价值流的参数。
  - $ \alpha $：优势流的参数。
  - $ |\mathcal{A}| $：动作空间大小。

**动机**：
- 分离状态价值和动作优势可以更高效地学习状态的重要性，尤其在动作选择对回报影响较小的环境中。
- 减去平均优势值提高训练稳定性，确保优势函数的平均值为零。

2. 网络结构

**Dueling DQN 的网络结构**：
- **共享特征提取层**：通常是卷积层（对于图像输入，如 Atari 游戏）或全连接层（对于向量输入，如 `RollingBall` 环境）。这些层提取状态的通用特征。
- **价值流**：输出单一标量 $ V(s) $，表示状态价值。
- **优势流**：输出与动作空间同维的向量 $ A(s, a) $，表示每个动作的优势。
- **组合层**：将价值流和优势流的结果组合，计算 Q 值。

**数学表示**：
- 输入：状态 $ s $（例如，4 维向量 `[x_position, y_position, x_velocity, y_velocity]`）。
- 共享特征提取：$ f(s; \theta) $，生成特征向量。
- 价值流：$ V(s; \theta, \beta) = W_v f(s; \theta) + b_v $，输出标量。
- 优势流：$ A(s, a; \theta, \alpha) = W_a f(s; \theta) + b_a $，输出向量。
- 最终 Q 值：
  $$
  Q(s, a) = V(s) + \left( A(s, a) - \frac{1}{|\mathcal{A}|} \sum_{a'} A(s, a') \right)
  $$

3. 损失函数

**损失函数**:
- Dueling DQN 使用与标准 DQN 相同的基于时间差分（TD）误差的损失函数：
  $$
  theta, \alpha, \beta) = \mathbb{E}_{(s, a, r, s', d) \sim D} \left[ \left( r + \gamma (1-d) \max_{a'} Q(s', a'; \theta^-, \alpha^-, \beta^-) - Q(s, a; \theta, \alpha, \beta) \right)^2 \right]
  $$
  - $ r $：即时奖励。
  - $ \gamma $：折扣因子（例如，代码中为 0.99）。
  - $ d $：终止标志（done）。
  - $ D $：经验回放缓冲区。
  - $ \theta^-, \alpha^-, \beta^- $：目标网络参数。
- 目标 Q 值由目标网络计算，目标网络定期更新以稳定训练：
  $$
  \theta^- \leftarrow \tau \theta + (1 - \tau) \theta^-
  $$
  - $ \tau $：软更新参数（例如，代码中为 0.001）。

**与标准 DQN 的区别**：
- 损失函数作用于分解后的 Q 值（通过价值流和优势流计算）。
- 分解结构有助于减少 Q 值过估计，因为状态价值 $ V(s) $ 提供了一个稳定的基线。

#### 4. 与标准 DQN 的区别

| **方面**            | **标准 DQN**                              | **Dueling DQN**                              |
|---------------------|------------------------------------------|---------------------------------------------|
| **Q 值估计**        | 直接估计 $ Q(s, a) $                 | 分解为 $ V(s) $ 和 $ A(s, a) $          |
| **网络输出**        | 每个动作的 Q 值                         | 状态价值 $ V(s) v 和动作优势 $ A(s, a) $ |
| **泛化性**          | 较差，尤其在动作相似时                 | 更好，独立学习状态价值                      |
| **稳定性**          | 可能出现 Q 值过估计                    | 通过分解减少过估计                          |
| **计算效率**        | 在大动作空间中效率较低                 | 通过分解提高效率                            |

**具体差异**：
- **动作无关性**：Dueling DQN 能更好地处理动作选择对回报影响较小的状态，通过学习 $ V(s) $ 减少冗余计算。
- **过估计缓解**：分解结构使 Q 值估计更稳定，尤其在结合 Double DQN 时。
- **效率提升**：在动作空间较大时（如 100 个动作），Dueling DQN 通过单一状态价值和相对优势值减少计算负担。
---

#### 同时拟合状态价值V(s)和动作优势A(s,a)，并通过组合公式计算Q值

In [8]:
class VA_net(torch.nn.Module):
    """_summary_

    Args:
        torch (_type_): _description_
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(VA_net, self).__init__()
        # 共享网络部分
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2= torch.nn.Linear(hidden_dim, hidden_dim)
        # 输出每个动作的优势值A(s,a)，维度为动作空间的大小
        self.fc_A = torch.nn.Linear(hidden_dim, output_dim)
        # 输出状态价值V(s),维度为1
        self.fc_V = torch.nn.Linear(hidden_dim, 1)
        self._init_weights()
    
    def _init_weights(self):
        """使用Kaiming初始化权重，适合激活函数为Relu"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                # 使用 Kaiming 均匀初始化，指定 nonlinearity='relu'
                nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')
                # 偏置置零
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
        
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        A = self.fc_A(F.relu(x))
        V = self.fc_V(F.relu(x))
        Q = V + A - A.mean().item()
        return Q

#### Dueling DQN：

In [9]:
class DuelingDON(DQN):
    def __init__(self, state_dim, hidden_dim, action_dim, action_range, lr, gamma, epsilon, device, tau=0.001, seed=None):
        super().__init__(state_dim, hidden_dim, action_dim, action_range, lr, gamma, epsilon, device, tau, seed)
        
        self.q_net = VA_net(state_dim, hidden_dim, action_range).to(self.device)
        self.target_q_net = VA_net(state_dim, hidden_dim, action_range).to(self.device)
        self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr = lr)


#### Reinforce & Actor-Critic:

**什么是 REINFORCE？**  
REINFORCE 是一种策略梯度方法，通过直接调整策略参数来最大化累积奖励。它使用蒙特卡洛方法，收集完整的情节（状态-动作-奖励序列），然后根据回报计算梯度更新策略。适合处理大型状态空间，但因梯度方差高，可能学习较慢。  

**什么是 Actor-Critic？**  
Actor-Critic 结合策略（Actor）和价值函数（Critic），Actor 选择动作，Critic 评估动作质量。使用时序差分学习，样本效率更高，适合复杂环境如机器人控制。包括多种变体，如 A2C 和 DDPG，特别在连续动作空间中表现优异。  

**两者的对比**  
- **效率：** Actor-Critic 通常比 REINFORCE 更高效，因其使用 Critic 降低梯度方差。  
- **适用场景：** REINFORCE 适合简单任务，Actor-Critic 更适合复杂、连续动作的任务。   

---

#### REINFORCE 算法详解  
REINFORCE（也被称为Monte Carlo Policy Gradient）是一种通过调整策略参数 $\theta$ 来优化策略 $\pi_\theta(a \mid s)$ 的方法。在强化学习中，策略 $\pi_\theta(a \mid s)$ 定义了在状态 $s$ 下选择动作 $a$ 的概率。其目标是最大化期望累积奖励 $J(\theta)$，数学上表示为：

$$
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} [R(\tau)]
$$

- $\tau = (s_0, a_0, s_1, a_1, \dots, s_T)$ 是一个轨迹（trajectory），由状态和动作序列组成。
- $R(\tau) = \sum_{t=0}^T \gamma^t r_t$ 是轨迹的总回报，其中 $\gamma \in [0, 1]$ 是折扣因子，$r_t$ 是每一步的奖励。

---

2. **策略梯度定理**
REINFORCE的核心是**策略梯度定理（Policy Gradient Theorem）**，它提供了目标函数 $J(\theta)$ 关于策略参数 $\theta$ 的梯度表达式：

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t \mid s_t) \cdot G_t \right]
$$

- $G_t = \sum_{k=t}^T \gamma^{k-t} r_k$ 是从时间步 $t$ 到轨迹结束的总回报。
- $\nabla_\theta \log \pi_\theta(a_t \mid s_t)$ 是策略对动作 $a_t$ 在状态 $s_t$ 下概率的对数梯度。

这个定理告诉我们，沿着梯度方向调整 $\theta$ 可以增加期望回报。

---

3. **蒙特卡洛估计**
由于上述期望无法解析计算，REINFORCE使用**蒙特卡洛采样（Monte Carlo Sampling）**来估计梯度：
- 代理（agent）根据当前策略 $\pi_\theta$ 与环境交互，生成多个轨迹。
- 对于每个轨迹，计算每个时间步的回报 $G_t$。
- 使用样本估计梯度：

$$
\hat{\nabla}_\theta J(\theta) = \frac{1}{N} \sum_{i=1}^N \sum_{t=0}^{T_i} \nabla_\theta \log \pi_\theta(a_{i,t} \mid s_{i,t}) \cdot G_{i,t}
$$

- $N$ 是采样的轨迹数量。
- 然后通过梯度上升更新策略参数：

$$
\theta \leftarrow \theta + \alpha \hat{\nabla}_\theta J(\theta)
$$

其中 $\alpha$ 是学习率。

---

4. **降低方差的基线**
REINFORCE的一个主要问题是梯度估计的**高方差**，因为回报 $G_t$ 受环境随机性影响较大。为了减少方差，可以引入一个**基线（Baseline）**，通常是状态价值函数 $V(s_t)$，改进梯度估计为：

$$
\nabla_\theta J(\theta) = \mathbb{E} \left[ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t \mid s_t) \cdot (G_t - V(s_t)) \right]
$$

基线不会改变梯度的无偏性，但通过使回报居中（centering），显著降低方差，从而加速和稳定学习。

---

5. **算法流程**
REINFORCE的工作流程如下：
- **初始化**策略参数 $\theta$。
- 使用当前策略 $\pi_\theta$ 生成多个完整轨迹。
- 对于每个轨迹，计算每个时间步的回报 $G_t$。
- 使用样本估计策略梯度 $\hat{\nabla}_\theta J(\theta)v$。
- 更新参数：$\theta \leftarrow \theta + \alpha \hat{\nabla}_\theta J(\theta)$。
- 重复步骤2-5，直到策略收敛。

---

6. **数学特性**
- **无偏估计**：蒙特卡洛方法提供的梯度估计是无偏的。
- **高方差**：依赖完整轨迹的回报导致方差较大，收敛可能较慢。
- **收敛性**：在适当的学习率条件下（例如 $\alpha \to 0$，$\sum \alpha = \infty$，$\sum \alpha^2 < \infty$），REINFORCE能收敛到局部最优。

---

7. **优点与局限性**

**优点**：
- **无模型（Model-Free）**：不需要环境的动态模型。
- **支持随机策略**：通过概率性动作选择实现天然的探索。
- **函数逼近**：可结合神经网络处理大状态空间。

**局限性**：
- **高方差**：梯度估计不稳定，学习效率低。
- **样本效率低**：需要完整轨迹，适合短时域任务，长时域任务表现不佳。
- **在线策略（On-Policy）**：每次更新需要重新采样，计算成本高。

---  

#### Actor-Critic 算法详解  
Actor-Critic 方法结合策略梯度和价值函数，首次由 Barto, Sutton 和 Anderson 在 1983 年提出，是 RL 中的混合方法。它由两个主要组件组成：  
- **Actor：** 负责根据当前策略 $ \pi(a \mid s, \theta) $ 选择动作，更新策略以最大化期望回报。  
- **Critic：** 评估 Actor 的动作质量，估计价值函数（如状态价值 $ v(s) $ 或动作价值 $ q(s, a) $），通常使用时序差分（TD）学习。  

- **核心原理：**  
  - Actor-Critic 通过 Critic 的反馈降低策略梯度的方差，相比 REINFORCE 更高效。  
  - Critic 的价值估计（如 TD 误差）作为 Actor 更新时的强化信号，公式为：  
    $$
    \delta_t = r_t + \gamma v(s_{t+1}) - v(s_t)
    $$
    其中 $ \delta_t $ 是 TD 误差，指导 Actor 更新。  

- **主要变体：**  
  - **Vanilla Actor-Critic：** 最简单形式，使用 Critic 的 TD 误差直接更新 Actor。  
  - **Advantage Actor-Critic (A2C)：** 引入优势函数 $ A(s, a) = q(s, a) - v(s) $，降低方差，更新公式为：  
    $$
    \nabla_\theta J(\theta) = \mathbb{E} \left[ \nabla_\theta \log \pi(a_t \mid s_t, \theta) \cdot A(s_t, a_t) \right]
    $$
    A2C 同时更新 Actor 和 Critic，适合探索性强的任务。  
  - **Asynchronous Advantage Actor-Critic (A3C)：** 使用多代理并行学习，提升稳定性和样本效率。  
  - **Deep Deterministic Policy Gradient (DDPG)：** 针对连续动作空间的离策略方法，结合 DQN 和 Actor-Critic。  
  - **Soft Actor-Critic (SAC)：** 最大化期望回报和策略熵，鼓励探索，适合高维连续任务。  

- **优势：**  
  - 样本效率高，因使用 TD 学习无需完整情节。  
  - 结合策略和价值方法，适合复杂环境如机器人控制和游戏 AI。  
  - 多种变体适应不同场景，特别在连续动作空间中表现优异。  

- **局限：**  
  - 实现复杂度较高，需要同时训练 Actor 和 Critic，调参难度大。  
  - Critic 的价值估计可能不准确，影响 Actor 的更新。  

Actor-Critic 是政策梯度和价值方法的优雅结合，解决了 REINFORCE 的高方差问题。  

#### REINFORCE 与 Actor-Critic 的对比  

以下表格总结两者的关键差异：  

| **特性**               | **REINFORCE**                          | **Actor-Critic**                       |
|-----------------------|---------------------------------------|---------------------------------------|
| **估计方法**           | 蒙特卡洛方法，需完整情节              | 时序差分学习，可用部分情节             |
| **梯度方差**           | 高，需基线降低                        | 低，Critic 提供基线或优势函数          |
| **样本效率**           | 较低，依赖完整序列                    | 较高，适合长序列任务                  |
| **学习速度**           | 较慢，因高方差可能不稳定              | 较快，Critic 反馈加速收敛              |
| **适用场景**           | 简单任务，离散动作空间                | 复杂任务，连续动作空间（如机器人控制） |
| **实现复杂度**         | 较低，单策略更新                      | 较高，需同时更新 Actor 和 Critic       |

- **效率对比：** Actor-Critic 因 Critic 的价值估计降低方差，通常比 REINFORCE 更高效，尤其在长序列或高维任务中。  
- **适用场景：** REINFORCE 适合初学者理解策略梯度，Actor-Critic 更适合现代 RL 应用，如深度强化学习中的游戏 AI 和自动驾驶。  
- **争议：** 一些研究（如 [Actor-critic methods — Mastering Reinforcement Learning](https://gibberblot.github.io/rl-notes/single-agent/actor-critic.html)）指出，Actor-Critic 的优势依赖实现细节，REINFORCE 在某些低维离散任务中可能更稳定。  

#### 应用与未来方向  
- **REINFORCE：** 常用于教学和简单任务，如迷宫导航，但因样本效率低，在现代 RL 中使用较少。  
- **Actor-Critic：** 广泛应用于深度 RL，如 AlphaGo 的策略网络训练，特别在连续动作空间中表现优异，如机器人控制和游戏 AI。  
- 未来方向包括改进 Actor-Critic 的样本效率（如离策略方法）和扩展到多代理 RL 系统。  

#### 结论  
REINFORCE 和 Actor-Critic 各有优势，REINFORCE 适合简单任务，Actor-Critic 更适合复杂环境。选择哪种算法需根据任务需求、动作空间和计算资源权衡。  

---




#### REINFORCE

##### 定义策略网络，用简单的MLP：

In [ ]:
class PolicyNet(torch.nn.Module):
    """
    策略网络，用于强化学习中的策略梯度方法（如 REINFORCE）。
    网络结构为三层 MLP（多层感知机），输入状态，输出动作概率分布。
    """
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        初始化策略网络。

        参数：
        - input_dim (int): 输入维度，即状态空间的维度。
        - hidden_dim (int): 隐藏层维度，控制网络容量。
        - output_dim (int): 输出维度，即动作空间的大小（离散动作）。

        网络结构：
        - fc1: 输入层 -> 隐藏层 1
        - fc2: 隐藏层 1 -> 隐藏层 2
        - fc3: 隐藏层 2 -> 输出层
        """
        super(PolicyNet, self).__init__()  # 调用父类构造函数，初始化 torch.nn.Module
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)  # 第一层线性变换：输入层 -> 隐藏层 1
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)  # 第二层线性变换：隐藏层 1 -> 隐藏层 2
        self.fc3 = torch.nn.Linear(hidden_dim, output_dim)  # 第三层线性变换：隐藏层 2 -> 输出层（动作概率）
        self._init_weights()  # 调用权重初始化方法，设置初始参数

    def _init_weights(self):
        """
        自定义权重初始化方法，确保网络初始行为适合强化学习任务。

        目标：
        - 隐藏层：使用 Kaiming 初始化，适合 ReLU 激活函数，保持激活值和梯度方差稳定。
        - 输出层：使用小方差正态分布，确保初始动作概率分布接近均匀，促进探索。
        """
        # 隐藏层 1 (fc1) 权重初始化
        # 使用 Kaiming 初始化（正态分布形式），适合 ReLU 激活函数
        # mode='fan_in'：方差计算基于输入神经元数量，避免激活值方差过大或过小
        # nonlinearity='relu'：指定激活函数为 ReLU，调整方差为 2/fan_in
        torch.nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
        
        # 隐藏层 1 (fc1) 偏置初始化
        # 初始化为 0.1（小正值），确保 ReLU 激活后更多神经元活跃（输出非 0）
        # 避免“神经元死亡”（ReLU 输出恒为 0），提升网络初始表达能力
        torch.nn.init.constant_(self.fc1.bias, 0)
        
        # 隐藏层 2 (fc2) 权重初始化
        # 同 fc1，使用 Kaiming 初始化，保持深层网络中激活值和梯度的稳定性
        torch.nn.init.kaiming_normal_(self.fc2.weight, mode='fan_in', nonlinearity='relu')
        
        # 隐藏层 2 (fc2) 偏置初始化
        # 同 fc1，设置为 0.1，增加神经元活跃性
        torch.nn.init.constant_(self.fc2.bias, 0)
        
        # 输出层 (fc3) 权重初始化
        # 使用小方差正态分布（均值 0，标准差 0.01），使初始权重接近 0
        # 这样，fc3 层的输出 z = Wx + b 接近 0，softmax(z) 接近均匀分布（1/output_dim）
        # 适合强化学习（如 REINFORCE）中初始探索需求，避免过早收敛到次优策略
        torch.nn.init.normal_(self.fc3.weight, mean=0, std=0.01)
        
        # 输出层 (fc3) 偏置初始化
        # 设置为 0，确保初始动作概率分布完全由权重决定，避免引入额外偏移
        torch.nn.init.zeros_(self.fc3.bias)

    def forward(self, x):
        """
        前向传播，计算状态对应的动作概率分布。

        参数：
        - x (torch.Tensor): 输入张量，形状为 (batch_size, input_dim)，表示一批状态。

        返回：
        - torch.Tensor: 动作概率分布，形状为 (batch_size, output_dim)，表示每个动作的概率。
        """
        # 第一层：线性变换 + ReLU 激活
        # 输入 x 的形状为 (batch_size, input_dim)
        # 输出形状为 (batch_size, hidden_dim)
        # ReLU(z) = max(0, z)，激活函数增加非线性，截断负值
        x = F.relu(self.fc1(x))
        
        # 第二层：线性变换 + ReLU 激活
        # 输入形状为 (batch_size, hidden_dim)
        # 输出形状为 (batch_size, hidden_dim)
        # 进一步增加网络深度和非线性表达能力
        x = F.relu(self.fc2(x))
        
        # 第三层：线性变换 + Softmax 激活
        # 输入形状为 (batch_size, hidden_dim)
        # 输出 z 的形状为 (batch_size, output_dim)，表示每个动作的未归一化得分（logits）
        # Softmax 将 logits 转换为概率分布，确保 sum(probs) = 1
        # 数学上：softmax(z)_i = exp(z_i) / sum(exp(z_j))
        # dim=1 表示在动作维度上归一化
        return F.softmax(self.fc3(x), dim=1)

#### REINFORCE:

In [ ]:
class REINFORCE(torch.nn.Module):
    """
    REINFORCE 算法实现，基于策略梯度方法，用于强化学习任务。
    该类包含策略网络（PolicyNet）和优化器，通过蒙特卡洛方法估计梯度并更新策略参数。
    """
    
    def __init__(self, state_dim, hidden_dim, action_range, learning_rate, gamma, device):
        """
        初始化 REINFORCE 算法。

        参数：
        - state_dim (int): 状态空间的维度（输入维度）。
        - hidden_dim (int): 策略网络隐藏层的维度，控制网络容量。
        - action_range (int): 动作空间的大小（离散动作数量，即输出维度）。
        - learning_rate (float): 学习率，用于优化器（Adam）。
        - gamma (float): 折扣因子，用于计算折扣回报，范围 [0, 1]。
        - device (torch.device): 计算设备（CPU 或 GPU，如 torch.device('cuda')）。
        """
        super().__init__()  # 调用父类 torch.nn.Module 的构造函数，初始化模块
        # 初始化策略网络 PolicyNet，用于参数化策略 π_θ(a|s)
        # state_dim -> hidden_dim -> action_range
        self.policynet = PolicyNet(state_dim, hidden_dim, action_range).to(device)
        # 使用 Adam 优化器优化策略网络参数
        # lr=learning_rate 指定学习率 α，用于梯度上升更新参数
        self.optimizer = torch.optim.Adam(self.policynet.parameters(), lr=learning_rate)
        # 折扣因子 γ，用于计算折扣回报 G_t
        self.gamma = gamma
        # 计算设备，确保张量和网络在同一设备上（CPU 或 GPU）
        self.device = device
        
    def take_action(self, state):
        """
        根据当前策略 π_θ(a|s) 从给定状态中采样动作。

        参数：
        - state (list or np.ndarray): 当前状态，通常是一个一维数组，形状为 (state_dim,)。

        返回：
        - int: 采样得到的动作索引（离散动作）。
        """
        # 将输入状态转换为 PyTorch 张量，并指定数据类型为浮点数
        # 示例：state = [0.5, 0.3, 1.0, -0.2] -> tensor([0.5, 0.3, 1.0, -0.2])
        state = torch.tensor(state, dtype=torch.float).to(self.device)
        # 增加批次维度，形状从 (state_dim,) 变为 (1, state_dim)
        # 神经网络（如 PolicyNet）通常期望输入是批量形式 (batch_size, input_dim)
        # 示例：tensor([0.5, 0.3, 1.0, -0.2]) -> tensor([[0.5, 0.3, 1.0, -0.2]])
        state = state.unsqueeze(0)
        # 通过策略网络计算动作概率分布 π_θ(a|s)，输出形状为 (1, action_range)
        # squeeze() 移除批次维度，形状变为 (action_range,)
        # 示例：tensor([[0.33, 0.33, 0.34]]) -> tensor([0.33, 0.33, 0.34])
        probs = self.policynet(state).squeeze()
        # 使用分类分布（Categorical Distribution）表示离散动作概率分布
        # torch.distributions.Categorical 需要一维概率向量，probs 的和为 1（由 softmax 保证）
        action_dist = torch.distributions.Categorical(probs)
        # 从概率分布中采样一个动作，action 是一个标量张量
        # 示例：若 probs=[0.33, 0.33, 0.34]，action 可能是 tensor(2)（以 0.34 的概率采样到动作 2）
        action = action_dist.sample()
        # 将张量转换为 Python 整数，方便传递给环境
        # 示例：tensor(2) -> 2
        return action.item()
    
    def update(self, transition_dict):
        """
        根据一条轨迹更新策略网络参数，使用 REINFORCE 算法的策略梯度方法。

        参数：
        - transition_dict (dict): 包含一条轨迹的数据，键包括：
            - 'rewards': 奖励列表 [r_0, r_1, ..., r_T]
            - 'states': 状态列表 [s_0, s_1, ..., s_T]
            - 'actions': 动作列表 [a_0, a_1, ..., a_T]

        数学原理：
        - 目标函数 J(θ) = E[R(τ)]，其中 R(τ) 是轨迹总回报
        - 策略梯度定理：∇_θ J(θ) = E[Σ_t ∇_θ log π_θ(a_t|s_t) * G_t]
        - G_t 是从时间步 t 开始的折扣回报：G_t = r_t + γ r_{t+1} + γ^2 r_{t+2} + ...
        """
        # 从字典中提取相关变量
        # 奖励列表 [r_0, r_1, ..., r_T]
        reward_list = transition_dict['rewards']
        # 状态列表 [s_0, s_1, ..., s_T]
        state_list = transition_dict['states']
        # 动作列表 [a_0, a_1, ..., a_T]
        action_list = transition_dict['actions']
        # 初始化折扣回报 G 为 0，用于递归计算 G_t
        G = 0
        # 清空优化器的梯度缓存，为本次更新准备
        # 避免上一次更新的梯度干扰
        self.optimizer.zero_grad()
        
        # 从轨迹最后一步（t=T）向前遍历到第一步（t=0）
        # 逆序计算折扣回报 G_t = r_t + γ r_{t+1} + γ^2 r_{t+2} + ...
        # reversed() 反转序列，例如 range(5) -> [4, 3, 2, 1, 0]
        for i in reversed(range(len(reward_list))):
            # 获取当前时间步 t=i 的奖励 r_t
            # 示例：reward_list=[1.0, 0.5, -0.2, 2.0, 3.0]，i=4 -> reward=3.0
            reward = reward_list[i]
            # 将当前状态 s_t 转换为 PyTorch 张量，并移动到指定设备
            # 形状为 (state_dim,)，例如 tensor([0.5, 0.3, 1.0, -0.2])
            state = torch.tensor(state_list[i], dtype=torch.float).to(self.device)
            # 增加批次维度，形状从 (state_dim,) 变为 (1, state_dim)
            # 适配 PolicyNet 的输入要求
            # 然后计算动作概率分布 π_θ(a|s_t)，输出形状为 (1, action_range)
            # squeeze() 移除批次维度，形状变为 (action_range,)
            # 示例：probs = tensor([0.33, 0.33, 0.34])
            probs = self.policynet(state.unsqueeze(0)).squeeze()
            # 获取当前时间步的动作 a_t
            # 示例：action_list=[0, 1, 2, 1, 0]，i=4 -> action=0
            action = action_list[i]
            # 计算动作 a_t 的对数概率 log π_θ(a_t|s_t)
            # probs[action] 获取第 action 个动作的概率
            # torch.log() 计算自然对数
            # 示例：若 probs[0]=0.33，则 log_prob=torch.log(0.33)≈-1.1086
            log_prob = torch.log(probs[action])
            # 递归计算折扣回报 G_t
            # G_t = r_t + γ G_{t+1}
            # 示例：若 gamma=0.99，G=0（初始），reward=3.0，则 G=3.0
            G = self.gamma * G + reward  
            # 计算当前时间步的损失
            # loss = -log π_θ(a_t|s_t) * G_t
            # 负号是因为 PyTorch 优化器执行梯度下降，而我们需要梯度上升
            # 梯度：∇_θ loss = -∇_θ log π_θ(a_t|s_t) * G_t，与策略梯度定理一致
            loss = -log_prob * G
            # 反向传播，计算损失对策略网络参数的梯度
            # 梯度会累积到 self.policynet.parameters() 的 .grad 属性中
            loss.backward()
        
        # 使用累积的梯度更新策略网络参数
        # Adam 优化器执行一步梯度下降：θ = θ - α * ∇_θ loss
        # 由于 loss 中有负号，实际执行的是梯度上升：θ = θ + α * ∇_θ J(θ)
        self.optimizer.step()

#### Actor-Critic:

#### 1. 数学原理

##### 1.1 强化学习的基本框架
强化学习的目标是通过与环境交互学习一个策略 $\pi(a|s)$，使代理（agent）在状态 $s$ 下选择动作 $a$，以最大化期望累积回报 $J(\pi)$。数学上，目标函数定义为：

$$
J(\pi) = \mathbb{E}_{\tau \sim \pi} [R(\tau)]
$$

- $\tau = (s_0, a_0, r_0, s_1, \dots, s_T)$ 是一条轨迹。
- $R(\tau) = \sum_{t=0}^T \gamma^t r_t$ 是轨迹的折扣累积回报，$\gamma \in [0, 1]$ 是折扣因子。
- $r_tv 是时间步 $t$ 的奖励，由环境提供。

在策略梯度方法中，策略 $\pi(a|s)$ 通常由参数 $\theta$ 参数化，记为 $\pi_\theta(a|s)$，目标变为优化 $\theta$：

$$
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} [R(\tau)]
$$

##### 1.2 策略梯度定理
Actor-Critic 的核心是策略梯度定理，它提供了 $J(\theta)$ 关于 $\theta$ 的梯度表达式：

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot G_t \right]
$$

- $G_t = \sum_{k=t}^T \gamma^{k-t} r_k$ 是从时间步 $t$ 开始的折扣回报。
- $\nabla_\theta \log \pi_\theta(a_t | s_t)$ 是对数似然梯度。

###### **推导过程**
1. 目标函数：
   $$
   J(\theta) = \int p(\tau | \theta) R(\tau) d\tau
   $$
   其中 $p(\tau | \theta) = p(s_0) \prod_{t=0}^T \pi_\theta(a_t | s_t) p(s_{t+1} | s_t, a_t)$是轨迹的概率。

2. 对 $\theta$ 求梯度：
   $$
   \nabla_\theta J(\theta) = \int \nabla_\theta p(\tau | \theta) R(\tau) d\tau
   $$

3. 使用对数导数技巧：
   $$
   \nabla_\theta p(\tau | \theta) = p(\tau | \theta) \nabla_\theta \log p(\tau | \theta)
   $$

4. 轨迹概率的对数：
   $$
   \log p(\tau | \theta) = \log p(s_0) + \sum_{t=0}^T \left( \log \pi_\theta(a_t | s_t) + \log p(s_{t+1} | s_t, a_t) \right)
   $$
   其中只有 $\pi_\theta(a_t | s_t)$ 依赖于 $\theta$，因此：
   $$
   \nabla_\theta \log p(\tau | \theta) = \sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t | s_t)
   $$

5. 代入梯度：
   $$
   \nabla_\theta J(\theta) = \int p(\tau | \theta) \left( \sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t | s_t) \right) R(\tau) d\tau
   $$
   $$
   = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot R(\tau) \right]
   $$

6. 因果性调整：
   - 由于 $a_t$ 只影响从 $t$ 开始的未来回报，因果性（causality）调整后，梯度变为：
     $$
     \nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot G_t \right]
     $$

##### 1.3 Actor-Critic 的改进
REINFORCE 算法直接使用蒙特卡洛方法估计 $G_t$，但其高方差会导致学习不稳定。Actor-Critic 引入价值函数（Critic）来估计 $G_t$，从而降低方差。

###### **价值函数和优势函数**
- **状态价值函数** $V^\pi(s)$，表示在策略 $\pi$ 下从状态 $s$开始的期望回报：
  $$
  V^\pi(s) = \mathbb{E}_{\pi} \left[ \sum_{t=0}^\infty \gamma^t r_t \mid s_0 = s \right]
  $$
- **动作价值函数** $Q^\pi(s, a)$，表示在状态 $s$ 下执行动作 $a$ 后的期望回报：
  $$
  Q^\pi(s, a) = \mathbb{E}_{\pi} \left[ \sum_{t=0}^\infty \gamma^t r_t \mid s_0 = s, a_0 = a \right]
  $$
- **优势函数** $A^\pi(s, a)$，衡量动作 $a$ 相较于平均水平的优劣：
  $$
  A^\pi(s, a) = Q^\pi(s, a) - V^\pi(s)
  $$

Actor-Critic 使用价值函数 $V(s)$ 或 $Q(s, a)$ 替代 $G_t$，梯度变为：
$$
\nabla_\theta J(\theta) = \mathbb{E} \left[ \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot A(s_t, a_t) \right]
$$

###### **时序差分（TD）估计**
Critic 使用时序差分（TD）方法估计价值函数，例如：
$$
\delta_t = r_t + \gamma V(s_{t+1}) - V(s_t)
$$
- $\delta_t$ 是 TD 误差，近似 $A(s_t, a_t)$。
- 相比蒙特卡洛估计的 $G_t$（需要完整轨迹），TD 估计只需当前步和下一步，显著降低方差。

#### 2. 模型原理

##### 2.1 Actor-Critic 结构
Actor-Critic 由两个主要组件组成：
- **Actor**：策略网络 $\pi_\theta(a|s)$，负责选择动作并更新策略。
- **Critic**：价值网络 $V_\phi(s)$ 或 $Q_\phi(s, a)$，负责评估动作或状态的价值，提供反馈。

###### **Actor**
Actor 使用策略梯度更新：
$$
\nabla_\theta J(\theta) = \mathbb{E} \left[ \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot \delta_t \right]
$$
- $\delta_t = r_t + \gamma V_\phi(s_{t+1}) - V_\phi(s_t)$ 是 TD 误差，作为 $A(s_t, a_t)$ 的估计。
- 参数更新：
  $$
  \theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)
  $$
  其中 $\alpha$ 是学习率。

###### **Critic**
Critic 使用 TD 学习更新价值函数：
$$
V_\phi(s_t) \leftarrow V_\phi(s_t) + \beta \delta_t
$$
- $\beta$ 是 Critic 的学习率。
- 损失函数通常为 TD 误差的平方：
  $$
  \mathcal{L}(\phi) = \frac{1}{2} \mathbb{E} [\delta_t^2]
  $$
- 参数更新：
  $$
  \phi \leftarrow \phi - \beta \nabla_\phi \mathcal{L}(\phi)
  $$

##### 2.2 数学推导：Actor-Critic 梯度
1. **优势函数的引入**：
   - 策略梯度定理：
     $$
     \nabla_\theta J(\theta) = \mathbb{E} \left[ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot G_t \right]
     $$
   - 引入基线 $V(s_t)$：
     $$
     \nabla_\theta J(\theta) = \mathbb{E} \left[ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot (G_t - V(s_t)) \right]
     $$
   - $G_t - V(s_t)$ 是优势函数 $A(s_t, a_t)$ 的蒙特卡洛估计。

2. **使用 TD 误差**：
   - 一步 TD 估计：
     $$
     G_t \approx r_t + \gamma V(s_{t+1})
     $$
     因此：
     $$
     A(s_t, a_t) \approx r_t + \gamma V(s_{t+1}) - V(s_t) = \delta_t
     $$
   - 梯度变为：
     $$
     \nabla_\theta J(\theta) = \mathbb{E} \left[ \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot \delta_t \right]
     $$

3. **Critic 的更新**：
   - Critic 最小化 TD 误差：
     $$
     \mathcal{L}(\phi) = \frac{1}{2} \delta_t^2
     $$
   - 梯度：
     $$
     \nabla_\phi \mathcal{L}(\phi) = \delta_t \cdot \nabla_\phi (r_t + \gamma V_\phi(s_{t+1}) - V_\phi(s_t))
     $$
     $$
     = \delta_t \cdot (\gamma \nabla_\phi V_\phi(s_{t+1}) - \nabla_\phi V_\phi(s_t))
     $$

##### 2.3 双时间尺度
Actor-Critic 是一种双时间尺度算法：
- Critic 更新较快（学习率 $\beta > \alpha$），以快速收敛到准确的价值估计。
- Actor 更新较慢，依赖 Critic 的估计进行策略优化。

#### 3. 实现细节

##### 3.1 网络结构
- **Actor**：策略网络 $\pi_\theta(a|s)$，通常是一个神经网络：
  - 输入：状态 $s$，形状为 `(state_dim,)`。
  - 输出：动作概率分布，例如离散动作空间使用 `softmax`。
- **Critic**：价值网络 $V_\phi(s)$，也是一个神经网络：
  - 输入：状态 $s$。
  - 输出：标量，表示状态价值 $V(s)$。

### 3.2 伪代码
```plaintext
初始化策略参数 θ 和价值参数 φ
对于每个回合：
    初始化状态 s
    收集一条轨迹：
        对于每个时间步 t：
            根据 π_θ(a|s) 采样动作 a
            执行动作，获取 r, s'
            计算 TD 误差：δ_t = r + γ V_φ(s') - V_φ(s)
            更新 Critic：φ ← φ + β δ_t ∇_φ V_φ(s)
            更新 Actor：θ ← θ + α δ_t ∇_θ log π_θ(a|s)
            s ← s'
        直到回合结束
```

##### 3.3 变体
- **A2C（Advantage Actor-Critic）**：
  - 使用优势函数 $A(s, a) = Q(s, a) - V(s)$，更准确地估计动作价值。
- **A3C（Asynchronous A2C）**：
  - 多个代理并行收集轨迹，提升样本效率。
- **DDPG（Deep Deterministic Policy Gradient）**：
  - 适用于连续动作空间，结合 Actor-Critic 和确定性策略梯度。
- **SAC（Soft Actor-Critic）**：
  - 引入熵正则化，最大化回报和策略熵，增强探索。

#### 4. 优缺点

##### 4.1 优点
- **低方差**：相比 REINFORCE，Critic 提供的 TD 估计显著降低梯度方差。
- **在线学习**：无需完整轨迹，支持单步更新，适合长序列任务。
- **灵活性**：通过变体（如 A2C、SAC）适应不同任务（离散/连续动作）。

##### 4.2 缺点
- **偏倚**：Critic 的价值估计可能引入偏倚。
- **实现复杂**：需要同时训练 Actor 和 Critic，调参较复杂。
- **收敛性**：双时间尺度可能导致不稳定，需仔细调整学习率。

#### 5. 结论

Actor-Critic 算法通过结合策略梯度和价值函数方法，解决了 REINFORCE 的高方差问题，同时保持了在线学习的灵活性。其数学核心是策略梯度定理和 TD 学习，模型结构分为 Actor（策略）和 Critic（价值估计）。尽管实现复杂，但在复杂任务（如机器人控制、游戏 AI）中表现优异，是现代强化学习的重要算法。

In [ ]:
class ActorCritic(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_range, actor_lr, critic_lr, gamma, device):
        super().__init__()
        self.gamma = gamma
        self.device = device
        
        self.actor = PolicyNet(state_dim, hidden_dim, action_range).to(self.device)
        self.critic = Q_Net(state_dim, hidden_dim, action_range).to(self.device)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=actor_lr)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=critic_lr)
    
    def take_action(self, state):
        state = torch.tensor(state, dtype=torch.float).to(self.device)
        state = state.unsqueeze(0)
        probs = self.actor(state)
        action_dist = torch.distributions.Categorical(probs)
        action = action_dist.sample()
        return action.item()
    
    def update_actor_critic(self, transition_dict):
        states = torch.tensor(transition_dict['states'], dtype=torch.float).to(self.device)
        next_states = torch.tensor(transition_dict['next_states'], dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions']).view(-1, 1).to(self.device)
        next_actions = torch.tensor(transition_dict['next_actions']).view(-1, 1).to(self.device)
        rewards = torch.tensor(transition_dict['rewards'], dtype=torch.float).view(-1, 1).to(self.device).squeeze()
        dones = torch.tensor(transition_dict['dones'], dtype=torch.float).view(-1, 1).to(self.device).squeeze()
        
        # Critic Loss
        q_values = self.critic(states).gather(dim=1, index=actions).squeeze()
        next_q_values = self.critic(next_states).gather(dim=1, index=next_actions).squeeze()
        td_targets = rewards + self.gamma * next_q_values * (1-dones)
        # td_targets 中包含 actor 给出的 next_action，将其 detach 以确保只更新 critic 参数
        critic_loss = torch.mean(F.mse_loss(q_values, td_targets.detach()))
        # Actor Loss
        log_probs = torch.log(self.actor(states).gather(dim=1, index=actions))
        actor_loss = torch.mean(-log_probs * q_values.detach())

        # 更新参数
        self.actor_optimizer.zero_grad()
        self.critic_optimizer.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        self.actor_optimizer.step()
        self.critic_optimizer.step()

#### **Critic 损失计算**

Critic 的目标是估计状态-动作价值 $ Q(s, a) $，并通过时序差分（TD）学习更新参数，使其逼近真实价值。

1. **q_values**:
   - **计算**：`self.critic(states)` 是一个形状为 `(bsz, action_range)` 的张量，表示每个状态下所有动作的 Q 值。
   - **gather**：`actions` 是一个形状为 `(bsz, 1)` 的张量，表示实际执行的动作索引。`gather(dim=1, index=actions)` 沿着动作维度（`dim=1`）选择对应的 Q 值，输出形状为 `(bsz, 1)`。
   - **squeeze**：移除维度为 1 的轴，形状变为 `(bsz,)`。
   - **数学原理**：$ Q(s_t, a_t) = \text{Critic}(s_t)[a_t] $，表示状态 $ s_t $ 下动作 $ a_t $ 的价值估计。

2. **next_q_values**:
   - **计算**：`self.critic(next_states)` 是一个形状为 `(bsz, action_range)` 的张量，表示下一状态下所有动作的 Q 值。
   - **gather**：`next_actions` 是 Actor 网络对 `next_states` 预测的动作索引，形状为 `(bsz, 1)`。`gather(dim=1, index=next_actions)` 选择对应的 Q 值，输出形状为 `(bsz, 1)`。
   - **squeeze**：形状变为 `(bsz,)`。
   - **数学原理**：$ Q(s_{t+1}, a_{t+1}) = \text{Critic}(s_{t+1})[a_{t+1}] $，其中 $ a_{t+1} $ 是 Actor 对 $ s_{t+1} $ 预测的动作。

3. **td_targets**:
   - **公式**：$ \text{TD Target} = r_t + \gamma Q(s_{t+1}, a_{t+1}) (1 - \text{done}_t) $
   - **解释**：
     - `rewards` 是当前奖励 $ r_t $，形状为 `(bsz,)`。
     - `self.gamma * next_q_values` 计算未来折扣回报 $\gamma Q(s_{t+1}, a_{t+1})$，形状为 `(bsz,)`。
     - `(1 - dones)` 是一个掩码，形状为 `(bsz,)`，当 `dones[t] = 1`（回合结束）时，未来回报为 0；当 `dones[t] = 0` 时，考虑未来回报。
   - **输出**：`td_targets` 形状为 `(bsz,)`，表示每个时间步的 TD 目标值。

4. **critic_loss**:
   - **公式**：$ \text{Loss} = \frac{1}{bsz} \sum_{t=1}^{bsz} (Q(s_t, a_t) - \text{TD Target}_t)^2 $
   - **解释**：
     - `F.mse_loss(q_values, td_targets.detach())` 计算均方误差，逐元素计算 `(q_values - td_targets)^2`。
     - `td_targets.detach()` 从计算图中分离，避免梯度回传到 Actor 网络（因为 `next_actions` 是 Actor 预测的）。
     - `torch.mean` 取平均，得到标量损失。
   - **数学原理**：Critic 损失基于 TD 误差，目标是使 $ Q(s, a) $ 逼近 TD 目标，优化 Critic 参数 $\phi$。

#### **Actor 损失计算**

Actor 的目标是优化策略 $ \pi_\theta(a|s) $，使期望回报最大化。

1. **probs**:
   - **计算**：`self.actor(states)` 是一个形状为 `(bsz, action_range)` 的张量，表示动作概率分布 $ \pi_\theta(a|s) $。
   - **gather**：`actions` 是实际动作索引，`gather(dim=1, index=actions)` 选择实际动作的概率，输出形状为 `(bsz, 1)`。
   - **squeeze**：形状变为 `(bsz,)`。
   - **数学原理**：$ \pi_\theta(a_t|s_t) = \text{Actor}(s_t)[a_t] $，表示状态 $ s_t $ 下动作 $ a_t $ 的概率。

2. **log_probs**:
   - **计算**：`torch.log(probs)` 计算对数概率，形状为 `(bsz,)`。
   - **数学原理**：$ \log \pi_\theta(a_t|s_t) $，是策略梯度定理中的对数似然项。

3. **actor_loss**:
   - **公式**：$ \text{Loss} = \frac{1}{bsz} \sum_{t=1}^{bsz} -\log \pi_\theta(a_t|s_t) \cdot Q(s_t, a_t) $
   - **解释**：
     - `-log_probs * q_values.detach()`：逐元素计算策略梯度的负值，形状为 `(bsz,)`。
     - `q_values.detach()` 从计算图中分离，避免梯度回传到 Critic 网络。
     - `torch.mean` 取平均，得到标量损失。
   - **数学原理**：根据策略梯度定理，梯度为：
     $$
     \nabla_\theta J(\theta) = \mathbb{E} \left[ \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot Q(s_t, a_t) \right]
     $$
     定义损失为 $-\log \pi_\theta(a_t|s_t) \cdot Q(s_t, a_t)$，其梯度即为策略梯度。

---

### 总结

- **Critic 损失**：
  - **目标**：使 $ Q(s, a) $ 逼近 TD 目标，基于均方误差。
  - **形状变化**：
    - `q_values` 和 `next_q_values`：`(bsz, action_range)` -> `(bsz, 1)`（`gather`）-> `(bsz,)`（`squeeze`）。
    - `td_targets`：`(bsz,)`。
    - `critic_loss`：标量。
- **Actor 损失**：
  - **目标**：优化策略 $ \pi_\theta(a|s) $，基于策略梯度。
  - **形状变化**：
    - `probs` 和 `log_probs`：`(bsz, action_range)` -> `(bsz, 1)`（`gather`）-> `(bsz,)`（`squeeze`）。
    - `actor_loss`：标量。
- **分离计算图**：
  - `td_targets.detach()` 和 `q_values.detach()` 确保 Critic 和 Actor 的梯度独立计算，避免相互干扰。

这些损失函数实现了 Actor-Critic 算法的双时间尺度更新，Critic 提供价值估计，Actor 优化策略。